In [12]:
!pip install pandas

import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize
from datetime import datetime, date, timedelta
import os.path 
from os import path

In [13]:
#returns weather json data from weatherstack api
def request_weather_json(key, date, location):
    params = {
        'access_key': key,
        'query': location,
        'historical_date': date,
        'hourly': '1',
        'interval': '1'
        }

    api_result = requests.get('https://api.weatherstack.com/historical', params)
    api_response = api_result.json()
    print(api_response['location'])
    return api_response

In [14]:
#Formats weather json data and outputs to detailed_weather_report.csv
def append_weather_data(json):
    json_historical = json_normalize(data=json['historical'][date], errors='ignore')
    detailed_weather_report = json_normalize(data=json_historical['hourly'][0], meta=['day'], meta_prefix='pre', errors='ignore')
    detailed_weather_report['location'] = json['location']['name']
    detailed_weather_report['date'] = date
    
    if path.exists("detailed_weather_report.csv"):
        detailed_weather_report.to_csv('detailed_weather_report.csv', mode='a', header=False)
    else:
        detailed_weather_report.to_csv('detailed_weather_report.csv')

In [15]:
def create_report():
    detailed_weather_df = pd.read_csv("detailed_weather_report.csv")
    weather_df = detailed_weather_df[detailed_weather_df['Unnamed: 0'].between(9,17)]
    weather_df['precip'].replace({0:np.nan}, inplace=True)
    weather_df = weather_df.groupby(['location','date']).agg({'precip':'count', 'temperature': 'max'}).reset_index().rename(columns={'precip':'Hours Rain', 'temperature':'Max Temp'})
    weather_df.to_csv('weather_report.csv')
    weather_df

In [28]:
detailed_weather_df = pd.read_csv("detailed_weather_report.csv")
weather_df = detailed_weather_df[detailed_weather_df['Unnamed: 0'].between(9,17)]
weather_df['precip'].replace({0:np.nan}, inplace=True)
weather_df = weather_df.groupby(['location','date']).agg({'precip':'count', 'temperature': 'max'}).reset_index().rename(columns={'precip':'RH', 'temperature':'TMP'})
weather_df['RH%'] = round(weather_df['RH']/9*100,0)
weather_df.pivot(index='location', columns='date', values=['RH%', 'TMP'])
#weather_df

RH%                                                         \
date     2020-01-05 2020-01-06 2020-01-07 2020-01-19 2020-01-20 2020-01-21   
location                                                                     
Belfast         0.0       33.0       78.0       78.0       89.0      100.0   
Glasgow        11.0       78.0      100.0       67.0       89.0      100.0   
Leeming         0.0       33.0       11.0       67.0       56.0       89.0   
London          0.0       11.0        0.0      100.0      100.0      100.0   

                TMP                                                         
date     2020-01-05 2020-01-06 2020-01-07 2020-01-19 2020-01-20 2020-01-21  
location                                                                    
Belfast        10.0        9.0       14.0        2.0        3.0        7.0  
Glasgow         8.0        9.0       12.0        2.0        3.0        6.0  
Leeming         8.0        8.0       11.0        2.0        1.0        4.0  
London          9.0        8.0       11.0        6.0        7.0        8.0

In [16]:
today = datetime.now().date()
week_start = today - timedelta(days=today.weekday()+1)

# this will give you a list containing all of the dates
week_days = [(week_start + timedelta(days=x)).isoformat() for x in range((today-week_start).days + 1)]

week_days

['2020-01-19', '2020-01-20', '2020-01-21']

In [20]:
key = '8c5dad538cf97d8347dbfda0e9369f25'
#dates = ['2020-01-01', '2020-01-02']
dates = week_days
#regions = ['Glasgow, UK', 'leeming, UK', 'London, UK', 'Belfast']
regions_dict = {'RegionID':[1, 3, 6, 8],'Region':['Glasgow, UK', 'leeming, UK', 'London, UK', 'Belfast']}

for region in regions_dict['Region']:
    for date in dates:    
        json = request_weather_json(key, date, region)
        append_weather_data(json)
        
create_report()

/srv/conda/envs/notebook/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
